In [2]:
from os import listdir
from os.path import isdir, join
import os
import glob, re
import random
import numpy as np
import matplotlib.pyplot as plt


In [3]:
dataset_path = os.getcwd() + "\\DataSet\\"
for name in listdir(dataset_path):
        if isdir(join(dataset_path, name)):
            print(name)
print(dataset_path)

Azimuth
Doppler
c:\Users\Iman\Desktop\Stage\code\cross_feature\DataSet\


In [4]:
# Create a dictionnary for each input type, store them in a list
type_list = [name for name in listdir(dataset_path) if isdir(join(dataset_path, name))]
print(type_list)

class_list = ['idle','presence','object_moved']
print(class_list)

['Azimuth', 'Doppler']
['idle', 'presence', 'object_moved']


In [5]:
# Settings
perc_keep_samples = 1.0 # 1.0 is keep all samples
val_ratio = 0.15
test_ratio = 0.15 


In [6]:
# Return float from csv file
def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

# Split the initial string in separated float substrings
def natural_keys(text):
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

In [7]:
# Create dictionnaries containing each samples that will be used for each class recognition
idle_dict = {}
idle_dict['Azimuth'] = glob.glob(dataset_path + 'Azimuth' + "\\" + 'idle' + "/*.csv")
idle_dict['Doppler'] = glob.glob(dataset_path + 'Doppler' + "\\" + 'idle' + "/*.csv")
print(len(idle_dict['Azimuth']),len(idle_dict['Doppler']))

presence_dict = {}
presence_dict['Azimuth'] = glob.glob(dataset_path + 'Azimuth' + "\\" + 'presence' + "/*.csv")
presence_dict['Doppler'] = glob.glob(dataset_path + 'Doppler' + "\\" + 'presence' + "/*.csv")
print(len(presence_dict['Azimuth']),len(presence_dict['Doppler']))

object_dict = {}
object_dict['Azimuth'] = glob.glob(dataset_path + 'Azimuth' + "\\" + 'object_moved' + "/*.csv")
object_dict['Doppler'] = glob.glob(dataset_path + 'Doppler' + "\\" + 'object_moved' + "/*.csv")
print(len(object_dict['Azimuth']),len(object_dict['Doppler']))

360 360


360 360
360 360


In [8]:
# Read CSV Function
def readCSV(filename):
    data = np.loadtxt(filename, dtype = np.float32, delimiter = ' ')
    return data

In [9]:
# MIN-MAX Function
def min_max(list):
    min = 5000
    max = 0
    count = 0
    for i in range(len(list)):
        heatmap = list[i]
        min = np.minimum(min, np.min(heatmap))
        max = np.maximum(max, np.max(heatmap))
        count += 1            
    return min, max


In [10]:
# Create true output vector containing the class type for each sample
y = [0]*min(len(idle_dict['Azimuth']),len(idle_dict['Doppler'])) + [1]*min(len(presence_dict['Azimuth']),len(presence_dict['Doppler'])) + [2]*min(len(object_dict['Azimuth']),len(object_dict['Doppler']))
print(len(y))

1080


In [11]:
print(readCSV(idle_dict['Azimuth'][0]).shape)
print(readCSV(idle_dict['Doppler'][0]).shape)


(256, 31)
(31, 256)


In [12]:
# Read features csv and make a feature array of it
n_samples = len(y)
az_features = np.zeros((n_samples,readCSV(idle_dict['Azimuth'][0]).shape[0],readCSV(idle_dict['Azimuth'][0]).shape[1]),dtype=np.float32)
dop_features = np.zeros((n_samples,readCSV(idle_dict['Doppler'][0]).shape[0],readCSV(idle_dict['Doppler'][0]).shape[1]),dtype=np.float32)

# Azimuth
az_idle_n = len(idle_dict['Azimuth'])
az_pres_n = len(presence_dict['Azimuth'])
az_obj_n = len(object_dict['Azimuth'])
for i in range(az_idle_n):
    az_features[i] = readCSV(idle_dict['Azimuth'][i])
for i in range(az_idle_n, az_idle_n + az_pres_n):
    az_features[i] = readCSV(presence_dict['Azimuth'][i - az_idle_n])
for i in range(az_idle_n + az_pres_n, az_idle_n + az_pres_n + az_obj_n):
    az_features[i] = readCSV(object_dict['Azimuth'][i - az_idle_n - az_pres_n])

# Doppler
dop_idle_n = len(idle_dict['Doppler'])
dop_pres_n = len(presence_dict['Doppler'])
dop_obj_n = len(object_dict['Doppler'])
for i in range(dop_idle_n):
    dop_features[i] = readCSV(idle_dict['Doppler'][i])
for i in range(dop_idle_n, dop_idle_n + dop_pres_n):
    dop_features[i] = readCSV(presence_dict['Doppler'][i - dop_idle_n])
for i in range(dop_idle_n + dop_pres_n, dop_idle_n + dop_pres_n + dop_obj_n):
    dop_features[i] = readCSV(object_dict['Doppler'][i - dop_idle_n - dop_pres_n])

# Normalizing arrays
min_az, max_az = min_max(az_features)
az_features = np.subtract(az_features,min_az)
az_features = np.divide(az_features,max_az-min_az)
print(min_az,max_az)

min_dop, max_dop = min_max(dop_features)
dop_features = np.subtract(dop_features,min_dop)
dop_features = np.divide(dop_features,max_dop-min_dop)
print(min_dop,max_dop)

print(az_features.shape)
print(dop_features.shape)

0.0 24825.0
1266.0 5063.0
(1080, 256, 31)
(1080, 31, 256)


In [12]:
# Associate feature array with true output vector
features_y = list(zip(az_features,dop_features, y))
print(features_y)

# Shuffle
random.shuffle(features_y)

# Extract shuffled lists
az_features, dop_features, y = zip(*features_y)

print("Shuffled y:",y)

[(array([[0.02940584, 0.00696878, 0.01623364, ..., 0.06006042, 0.06376637,
        0.05192346],
       [0.07681772, 0.03037261, 0.02988922, ..., 0.12471299, 0.13748237,
        0.11891238],
       [0.07415912, 0.03238671, 0.02952669, ..., 0.11645519, 0.12733132,
        0.11121853],
       ...,
       [0.00080564, 0.00128902, 0.00140987, ..., 0.00108761, 0.00068479,
        0.00032226],
       [0.00338369, 0.00378651, 0.0033434 , ..., 0.00092649, 0.00064451,
        0.00221551],
       [0.00326284, 0.0064854 , 0.00753273, ..., 0.00946626, 0.00725076,
        0.00338369]], dtype=float32), array([[0.17803529, 0.21306294, 0.20200159, ..., 0.15591256, 0.16855413,
        0.16513036],
       [0.22175401, 0.20963919, 0.20173821, ..., 0.13299973, 0.1316829 ,
        0.18224914],
       [0.19015011, 0.19673426, 0.2272847 , ..., 0.12799579, 0.14511457,
        0.17671846],
       ...,
       [0.22359757, 0.19910456, 0.13457993, ..., 0.18936002, 0.19304714,
        0.19094022],
       [0.2048986

In [13]:
# Only keep the specified number of samples (shorter extraction / training)
az_features = az_features[:int(len(az_features) * perc_keep_samples)]
print('Number of Azimuth samples kept:',len(az_features))

dop_features = dop_features[:int(len(dop_features) * perc_keep_samples)]
print('Number of Doppler samples kept:',len(dop_features))


Number of Azimuth samples kept: 1080
Number of Doppler samples kept: 1080


In [14]:
# Calculate validation and test set sizes
val_set_size = int(min(len(az_features),len(dop_features)) * val_ratio)
test_set_size = int(min(len(az_features),len(dop_features)) * test_ratio)
print('Validation set size:',val_set_size)
print('Testing set size:',test_set_size)

Validation set size: 162
Testing set size: 162


In [15]:
# Break dataset apart into train, validation, and test sets
az_val_feat = az_features[:val_set_size]
az_test_feat = az_features[val_set_size:(val_set_size + test_set_size)]
az_train_feat = az_features[(val_set_size + test_set_size):]

dop_val_feat = dop_features[:val_set_size]
dop_test_feat = dop_features[val_set_size:(val_set_size + test_set_size)]
dop_train_feat = dop_features[(val_set_size + test_set_size):]

print(len(dop_train_feat))
print(len(az_val_feat))
print(len(dop_test_feat))

756
162
162


In [16]:
# Break y apart into train, validation, and test sets
y_val = y[:val_set_size]
y_test = y[val_set_size:(val_set_size + test_set_size)]
y_train = y[(val_set_size + test_set_size):]

print(len(y_train),y_train)
print(len(y_val),y_val)
print(len(y_test),y_test)

756 (0, 0, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 0, 2, 2, 0, 0, 0, 1, 0, 2, 1, 2, 0, 2, 0, 0, 2, 2, 1, 0, 2, 1, 1, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 1, 1, 2, 0, 1, 1, 0, 1, 1, 2, 0, 2, 1, 0, 1, 0, 2, 1, 1, 2, 2, 2, 0, 1, 0, 1, 1, 1, 0, 0, 2, 0, 0, 2, 1, 0, 2, 2, 0, 2, 2, 1, 2, 0, 0, 2, 0, 1, 0, 0, 1, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 1, 1, 2, 1, 1, 0, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 1, 2, 0, 1, 0, 0, 1, 2, 1, 1, 2, 2, 0, 0, 1, 0, 2, 1, 1, 2, 0, 1, 2, 1, 0, 0, 2, 1, 1, 2, 1, 1, 0, 2, 2, 2, 0, 1, 1, 2, 2, 1, 2, 2, 0, 1, 1, 0, 1, 0, 2, 1, 0, 1, 2, 2, 2, 0, 2, 2, 2, 1, 0, 1, 1, 2, 2, 0, 1, 2, 1, 0, 0, 1, 2, 2, 1, 1, 1, 0, 2, 1, 2, 2, 0, 2, 1, 1, 2, 0, 2, 0, 0, 1, 0, 1, 2, 0, 0, 1, 2, 1, 2, 0, 1, 0, 0, 2, 0, 1, 1, 0, 2, 2, 1, 0, 2, 0, 0, 2, 2, 2, 0, 1, 1, 1, 1, 0, 0, 1, 2, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, 1, 2, 1, 1, 0, 1, 0, 1, 1, 0, 2, 2, 1, 0, 0, 2, 2, 2, 0, 2, 2, 1, 1, 2, 0, 1, 0, 2, 2, 2, 1, 0, 0, 0, 2, 2, 1, 1, 0, 2, 0, 1, 0, 2, 1, 1, 1, 2, 1, 2, 0, 2, 1,

In [17]:
# Save features and truth vector (y) sets to disk
feature_sets_file='all_targets_cross_' + str(n_samples) + '.npz'
np.savez(feature_sets_file,
        x_train_az=az_train_feat,
        x_train_dop=dop_train_feat,
        y_train=y_train,
        x_val_az=az_val_feat,
        x_val_dop=dop_val_feat,
        y_val=y_val,
        x_test_az=az_test_feat,
        x_test_dop=dop_test_feat,
        y_test=y_test)


In [18]:
# TEST: Load features
feature_set = np.load(feature_sets_file)
print(feature_set.files)

['x_train_az', 'x_train_dop', 'y_train', 'x_val_az', 'x_val_dop', 'y_val', 'x_test_az', 'x_test_dop', 'y_test']


In [20]:
print(feature_set['x_train_az'].shape)
print(feature_set['x_train_dop'].shape)

(756, 256, 31)
(756, 31, 256)
